In [1]:
# automatically reload dependencies and repository content so that kernel need not be restarted
%load_ext autoreload
%autoreload 2

In [2]:
import torch


device = None
if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device("cuda:0")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using GPU


In [3]:
from image_dataset import ImageDataset
from data_augmentation import DataAugmentation

dataAugmentation = DataAugmentation()
dataset = ImageDataset(dataAugmentation = dataAugmentation)

In [4]:
trainIds = range(1,19001)

In [5]:
from data_balancer import DataBalancer

dataBalancer = DataBalancer()

trainIds = dataBalancer.balanceData(trainIds, 80000-19000, trainIds)

In [6]:
trainDataset = ImageDataset(trainIds, dataAugmentation=dataAugmentation)
valDataset = ImageDataset(range(19001,20001))

In [54]:
from reseption_net import ReseptionNet
from convolution_components import BasicConv2d, MaxPooling, IdentityConv2d

torch.cuda.empty_cache()

basicInceptionConfig = {
    "branches": [
        {
            "blocks": [
                {   
                    "convolution": BasicConv2d,
                    "outputChannelMultiplier": 1,
                    "kernelSize": 1,
                    "padding": 0,
                    "stride": 1,
                    "dilation": 1
                },
                {   
                    "convolution": MaxPooling,
                    "outputChannelMultiplier": 1,
                    "kernelSize": 4,
                    "padding": 0,
                    "stride": 4,
                    "dilation": 1
                }
            ]
        },
        {
            "blocks": [
                {
                    "convolution": BasicConv2d,
                    "outputChannelMultiplier": 1,
                    "kernelSize": 3,
                    "padding": 1,
                    "stride": 1,
                    "dilation": 1
                },
                {   
                    "convolution": MaxPooling,
                    "outputChannelMultiplier": 1,
                    "kernelSize": 4,
                    "padding": 0,
                    "stride": 4,
                    "dilation": 1
                }
            ]
        },
        {
            "blocks": [
                {
                    "convolution": BasicConv2d,
                    "outputChannelMultiplier": 1,
                    "kernelSize": 5,
                    "padding": 2,
                    "stride": 1,
                    "dilation": 1
                },
                {   
                    "convolution": MaxPooling,
                    "outputChannelMultiplier": 1,
                    "kernelSize": 4,
                    "padding": 0,
                    "stride": 4,
                    "dilation": 1
                }
            ]
        },
        {
            "blocks": [
                {
                    "convolution": BasicConv2d,
                    "outputChannelMultiplier": 1,
                    "kernelSize": 7,
                    "padding": 3,
                    "stride": 1,
                    "dilation": 1
                },
                {   
                    "convolution": MaxPooling,
                    "outputChannelMultiplier": 1,
                    "kernelSize": 4,
                    "padding": 0,
                    "stride": 4,
                    "dilation": 1
                }
            ]
        }
        
    ],
    "shortcut": {
        "convolution": IdentityConv2d,
        "kernelSize": 1,
        "padding": 0,
        "stride": 4,
        "dilation": 1
    }
}

channelReducerConfig = {
    "branches": [
        {
            "blocks": [
                {   
                    "convolution": BasicConv2d,
                    "outputChannelMultiplier": 0.1,
                    "kernelSize": 1,
                    "padding": 0,
                    "stride": 1,
                    "dilation": 1
                }
            ]
        }
    ],
    "shortcut": {
        "convolution": IdentityConv2d,
        "kernelSize": 1,
        "padding": 0,
        "stride": 1,
        "dilation": 1
    }
}
    
config = {
    "inceptions": [
        {
            "config": basicInceptionConfig,
            "amount": 2,
        },
        {
            "config": channelReducerConfig,
            "amount": 0,
        },
    ],
    "inChannels": 3,
    "inDimensions": (224, 224),
    "outputs": 14,
}

model = ReseptionNet(device, config)

inception 1 iteration 1 layer output dimensions 12 * 56 * 56 = 37632
inception 1 iteration 2 layer output dimensions 48 * 14 * 14 = 9408


In [ ]:
from train_model import train_model

model = train_model(
    model,
    trainDataset,
    valDataset,
    device,
    numberOfEpochs = 1
)

Epoch 1/1
----------
Training...

Progress: 0%
Progress: 10%
Progress: 20%
Progress: 30%


In [ ]:
torch.save(model.state_dict(), 'shallow_model.pkl')
torch.cuda.empty_cache()

In [ ]:
model.load_state_dict(torch.load("shallow_model.pkl"))

In [ ]:
import eval_model

decisionThreshold = 0.5

yHats, yTrues = eval_model.test_model(model, valDataset, device, decisionThreshold=decisionThreshold)
for metric in ['precision', 'recall', 'f1', 'accuracy']:
    print("{}: {}".format(metric, eval_model.get_metric(yTrues, yHats, metric)))

In [ ]:
from example_prediction_shower import showExamplePredictions

print("Validation images:")
print()
showExamplePredictions(model, valDataset, device, 5, decisionThreshold=decisionThreshold)

In [ ]:
print("Training images:")
print()
showExamplePredictions(model, trainDataset, device, 5, decisionThreshold=decisionThreshold)